### Imports

In [74]:
import importlib
import traceback
import os
import re
from tqdm import tqdm

import findatree.io as io
import findatree.photometric as photometric
import findatree.transformations as transformations

_ = importlib.reload(io)
_ = importlib.reload(photometric)
_ = importlib.reload(transformations)

### Definitions

In [75]:
# Define full paths to directories containing dtm, dsm, ortho and shape-files 
dir_names=[]
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\DSMs'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\DTMs'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\Orthos'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\Kronen'])

# Define directories where return hdf5s are stored
dir_hdf5 = r"C:\Data\lwf\processed\2020\hdf5"

# Optionally process only specific tnrs
include_tnrs = [
    11581,
    11601,
    12545,
]

### Processing

#### Prepare main loop

In [71]:
# Get all available tnr numbers in directory
paths = [os.path.join(dir_hdf5, name) for name in os.listdir(dir_hdf5) if os.path.splitext(name)[-1] == '.hdf5']

# Include only specified tnrs in main loop
if len(include_tnrs) > 0:
    paths_keep = []
    for tnr in include_tnrs:
        pattern_tnr = 'tnr' + str(tnr)
        paths_keep.extend([p for p in paths if bool(re.search(pattern_tnr, os.path.split(p)[-1], re.IGNORECASE)) ] )
    print('Following files will be included in main loop:')
    print(paths_keep)
else:
    paths_keep = paths

Following files will be included in main loop:
['C:\\Data\\lwf\\processed\\2020\\hdf5\\tnr11581.hdf5', 'C:\\Data\\lwf\\processed\\2020\\hdf5\\tnr11601.hdf5', 'C:\\Data\\lwf\\processed\\2020\\hdf5\\tnr12545.hdf5']


#### Run main loop

In [87]:
exception_traceback = []

for path in tqdm(paths_keep):
    
    try:
        
        ################# [1] Load channels and crowns group from hdf5
        
        data, params_data = io.load_hdf5(path, groups = ['channels'])
        channels, params_channels = data['channels'].copy(), params_data['channels'].copy()
        del data, params_data


        ################# [2] Load and save shapefile to correct tnr and save in hdf5 under group 'crowns_human'
        
        crowns_human, params_crowns_human = io.load_shapefile(dir_names, params_channels, verbose=False)
        photometric.crowns_add_features(
            channels,
            params_channels,
            crowns_human,
            params_crowns_human,
        )
        io.crowns_to_hdf5(crowns_human, params_crowns_human, dir_name = dir_hdf5)

    except:
        exception_traceback.append({
            'path': path,
            'traceback': traceback.format_exc(),
            })


print()
print('Done!')
print(f"{len(exception_traceback)} exceptions occured")

100%|██████████| 3/3 [00:01<00:00,  2.20it/s]


Done!
2 exceptions occured


In [89]:
for e in exception_traceback:
    print(e['path'])
    print('-->', e['traceback'])

C:\Data\lwf\processed\2020\hdf5\tnr11581.hdf5
--> Traceback (most recent call last):
  File "C:\Users\flori\AppData\Local\Temp\ipykernel_6540\3651721285.py", line 17, in <cell line: 3>
    photometric.crowns_add_features(
  File "C:\Repos\findatree\findatree\photometric.py", line 357, in crowns_add_features
    labelimg = transformations.polygons_to_labelimage(crowns['polygons'], params_crowns['shape'])
  File "C:\Repos\findatree\findatree\transformations.py", line 214, in polygons_to_labelimage
    rasterio.features.rasterize(shapes, out = labelimg)
  File "c:\Users\flori\Anaconda3\envs\findatree_v01\lib\site-packages\rasterio\env.py", line 387, in wrapper
    return f(*args, **kwds)
  File "c:\Users\flori\Anaconda3\envs\findatree_v01\lib\site-packages\rasterio\features.py", line 321, in rasterize
    raise ValueError('No valid geometry objects found for rasterize')
ValueError: No valid geometry objects found for rasterize

C:\Data\lwf\processed\2020\hdf5\tnr11601.hdf5
--> Traceback (